<h3> Segmenting and Clustering Neighborhoods in Toronto</h3>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

WEB SCRAPING

In [4]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web_texto= requests.get(wikipedia_link).text


Canada_dado = BeautifulSoup(web_texto,'xml')


EXTRAINDO DADOS DA WEBPAGE

In [5]:
table = Canada_dado.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []


for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

INTEGRANDO AS VIZINHANÇAS 

In [6]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


CRIANDO O DATAFRAME

In [7]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)

df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M1S,Scarborough,Agincourt
1,M3K,North York,CFB Toronto
2,M3L,North York,Downsview West
3,M3J,North York,"Northwood Park, York University"
4,M4L,East Toronto,India Bazaar
5,M5J,Downtown Toronto,"Toronto Islands, Union Station"
6,M4B,East York,"Woodbine Gardens, Parkview Hill"
7,M4K,East Toronto,Riverdale
8,M5X,Downtown Toronto,"First Canadian Place, Underground city"
9,M6R,West Toronto,"Parkdale, Roncesvalles"


In [8]:
df_toronto.shape

(77, 3)